In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def extract_product_details(soup):
    product_name = soup.find("span", {"id": "productTitle"})
    product_name = product_name.get_text(strip=True) if product_name else "No product name available"
    
    price = soup.find("span", {"class": "a-price-whole"})
    price = price.get_text(strip=True) if price else "Not Available"
    
    img = soup.find("img", {"id": "landingImage"})
    img_url = img['src'] if img else "No image found"
    
    description = soup.find("div", {"id": "feature-bullets"})
    description = description.get_text(strip=True) if description else "No description available"
    
    rating = soup.find("span", {"class": "a-icon-alt"})
    rating = rating.get_text(strip=True) if rating else "No rating"

    return {
        "Product Name": product_name,
        "Price": price,
        "Image URL": img_url,
        "Description": description,
        "Overall Rating": rating,
    }

def extract_reviews(soup):
    reviews = []
    review_elements = soup.find_all("li", {"data-hook": "review"})
    for review in review_elements:
        review_text = review.find("span", {"data-hook": "review-body"})
        if review_text:
            reviews.append(review_text.get_text(strip=True))
    return reviews

def scrape_amazon_product(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    response = requests.get(url, headers=headers)

    if response.status_code != 200:
        print(f"Failed to retrieve the webpage. Status code: {response.status_code}")
        return None

    soup = BeautifulSoup(response.content, "html.parser")
    product_details = extract_product_details(soup)
    reviews = extract_reviews(soup)
    
    product_details["Reviews"] = reviews
    return product_details

def save_to_csv(product_data):
    if product_data:
        reviews = product_data.pop("Reviews", [])
        product_df = pd.DataFrame([product_data])
        review_df = pd.DataFrame(reviews, columns=["Review Text"])
        combined_df = pd.concat([product_df, review_df], axis=1)
        
        filename = "product_data.csv"
        combined_df.to_csv(filename, index=False)
        print(f"Data saved to {filename}")

if __name__ == "__main__":
    product_url = "https://amzn.in/d/eVKIxwZ"
    product_data = scrape_amazon_product(product_url)

    if product_data:
        save_to_csv(product_data)
    else:
        print("No data to save.")

Data saved to product_data.csv
